In [2]:
#MODULES 
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import cv2 
import numpy as np
import os
from random import shuffle
from tqdm import tqdm

print(tf.__version__)

2.2.0


In [3]:
# SETTING DIRECTORIES AND CONSTANT VALUES 
TRAIN_DIR = 'C:\\Users\\Hp\\Desktop\\Wobot\\Kaggle1\\dogs-vs-cats\\train\\train'
TEST_DIR = 'C:\\Users\\Hp\\Desktop\\Wobot\\Kaggle1\\dogs-vs-cats\\test1\\test1'
IMAGE_SIZE = 64
ALPHA = 0.001 # Hyperparameter: Learning Rate alpha 

In [5]:
#ONE HOT ENCODING
# Cats [1,0]
# Dogs [0,1]
def img_label(img):
    label = img.split('.')[-3]
    if label == 'cat':
        return np.array([1,0])
    elif label == 'dog':
        return np.array([0,1])
    else:
        print("Invalid image ")
        return np.array([0,0])
  
# GIVE ONE HOT ENCODED ARRAY AND RETURNS 1 OR 0
def label_class_str(arr):
    if np.array_equal(arr, np.array([1,0])):
        return 1
    elif np.array_equal(arr, np.array([0,1])):
        return 0
    else:
        print("Something went wrong")

In [7]:
# HELPER FUNCTION TO SEE DATA 
def print_array_image(index, dataType = 'train'):
    np.set_printoptions(linewidth=200)
    if dataType == 'train':
        plt.imshow(train_data[index])
        print('Label :',train_labels[index])
        print('Numpy array:',train_data[index])
    elif dataType == 'test':
        plt.imshow(test_data[index])
        print(test_id[index])
        print('Numpy array:',test_data[index])
    else:
        print("Enter right type of data")

In [8]:
#CREATE TRAINING DATA 
# RETURNS A NUMPY ARRAY WITH NUMPY ARRAYS 
# ARR 1 CONTAINS THE IMAGE
# ARR 2 CONTAINS THE LABEL (ONE HOT)
def create_training_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = img_label(img)
        path = os.path.join(TRAIN_DIR, img)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE), interpolation = cv2.INTER_NEAREST)
        training_data.append([np.array(img), np.array(label)])
    shuffle(training_data)
    return np.array(training_data)

In [ ]:
# INITIALISE TRAINING DATA
training_data = create_training_data()
train_data = np.array([i[0] for i in training_data])
train_labels = np.array([i[1] for i in training_data])
train_str = np.array(['Dog' ])

In [ ]:
print(train_data.shape) # (25000,50,50)
print(train_labels.shape) # (2500,2)
print(len(train_labels))

In [ ]:
# PROCESSING TEST DATA 
def process_testing_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        img_id = img.split('.')[-2]
        path = os.path.join(TEST_DIR, img)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE), interpolation = cv2.INTER_NEAREST)
        testing_data.append([np.array(img), img_id])
    return np.array(testing_data)

In [ ]:
testing_data = process_testing_data()
test_data = np.array([i[0] for i in testing_data])
test_id = np.array([i[1] for i in testing_data])

In [ ]:
print(testing_data.shape) # (12500, 2)
print(test_data.shape) # (12500, 50, 50)

In [ ]:
# PRE PROCESSING STEP 
train_data  = train_data / 255.0
test_data = test_data / 255.0

In [ ]:
train_class = np.zeros((len(train_labels)))
print(train_class.shape)
for i in range(len(train_labels)):
    train_class[i] = rev_label(train_labels[i])

In [ ]:
# CROSS VALIDATION 
traind = np.expand_dims(train_data[5000:] , axis=-1) 
trainc = np.expand_dims(train_class[5000:], axis = -1)
trainl = np.expand_dims(train_labels[5000:], axis = -1)

testd = np.expand_dims(train_data[:5000] , axis=-1)
testc = np.expand_dims(train_class[:5000], axis = -1)
testl = np.expand_dims(train_labels[:5000], axis = -1)

In [ ]:
print(traind.shape)
print(trainc.shape)
print(testd.shape)
print(testc.shape)

In [ ]:
from tensorflow.keras import datasets, layers, models

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(50,50,1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(traind, trainc, epochs=25,
                    validation_data=(testd, testc))

In [ ]:
print(test_data.reshape((12500,50,50,1)).shape)
test_data1 = test_data.reshape((12500,50,50,1))
print(test_id.shape)

In [ ]:
print(test_data1.shape)

In [ ]:
pred = model.predict(test_data1)

In [ ]:
print(len(pred))

In [ ]:
print(type(pred))
print(test_id.shape)
print(test_id[12499])

In [ ]:
test_new = np.unique(test_id)
print(test_new.shape)   

In [ ]:
fig=plt.figure()
    
ind = 1
#y = fig.add_subplot(3,4,num+1)
orig = test_data[int(ind)]
data = test_data[int(ind)].reshape(IMAGE_SIZE,IMAGE_SIZE,1)
pred1 = (np.argmax(pred[int(i)-1]))
    
if pred1 == 0: str_label='Dog'
else: str_label='Cat'
     
    
plt.imshow(orig,cmap='gray')
plt.title(str_label)
print(ind)
#y.axes.get_xaxis().set_visible(False)
#y.axes.get_yaxis().set_visible(False)
#plt.show()

In [ ]:
with open('sub_try.csv', 'w') as f:
    
    for i in test_new:
        j = (np.argmax(pred[int(i)-1]))
        print(i, j)
        f.write(f"{i},{j}\n")
        
    


In [ ]:
with open('sub_try.csv', 'w') as f:
    f.write('id, label\n')

In [ ]:
print(len(pred))

In [ ]:
with open('sub_try.csv', 'w') as f:
    for i in test_id:
        img_id = int(i)
        j = np.argmax(pred[int(i)])
        f.write('{},{}\n'.format(img_id, j))
       


In [ ]:

fig = plt.figure()

for num, data in enumerate(testing_data[:12]):
    